In [14]:
from Data_Handler.DataReader import DataReader
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from impressions import Impressions
from hybrid import *
from tqdm import tqdm
from evaluator import evaluate
import pandas as pd
import numpy as np
# Read & split data
dataReader = DataReader()

In [15]:
target = dataReader.load_target()

URM = dataReader.load_augmented_binary_urm()
URM_aug,icm = dataReader.pad_with_zeros_ICMandURM(URM)

URM_train_aug, URM_validation = split_train_in_two_percentage_global_sample(URM_aug, train_percentage = 0.9)
URM_train_pow = dataReader.stackMatrixes(URM_train_aug)

>>> number of target users: 41116
4877


In [16]:
UserKNNCF = UserKNNCFRecommender(URM_train_aug)
UserKNNCF.fit()

#RP3beta_pow = RP3betaRecommender(URM_train_pow)
#RP3beta_pow.fit(alpha=0.3648761546066018,beta=0.5058870363874656, topK=480, normalize_similarity=True)

S_SLIM = SLIMElasticNetRecommender(URM_train_pow)
S_SLIM.fit()

EASE_R = EASE_R_Recommender(URM_train_pow)
EASE_R.fit()

UserKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 41629 (100.0%), 591.53 column/sec. Elapsed time 1.17 min
SLIMElasticNetRecommender: Processed 5866 (21.0%) in 5.00 min. Items per second: 19.55
SLIMElasticNetRecommender: Processed 11969 (42.8%) in 10.00 min. Items per second: 19.94
SLIMElasticNetRecommender: Processed 18063 (64.6%) in 15.00 min. Items per second: 20.07
SLIMElasticNetRecommender: Processed 24166 (86.4%) in 20.00 min. Items per second: 20.14
SLIMElasticNetRecommender: Processed 27968 (100.0%) in 20.77 min. Items per second: 22.44
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 6.27 min


In [17]:
RP3beta_aug = RP3betaRecommender(URM_train_aug)
RP3beta_aug.fit(alpha=0.6951524535062256,beta=0.39985511876562174, topK=82, normalize_similarity=True)

RP3betaRecommender: URM Detected 3461 (12.4%) items with no interactions.
RP3betaRecommender: Similarity column 27968 (100.0%), 3491.69 column/sec. Elapsed time 8.01 sec


In [21]:
RP3beta_pow = RP3betaRecommender(URM_train_pow)
RP3beta_pow.fit(alpha=0.3648761546066018,beta=0.5058870363874656, topK=480, normalize_similarity=True)

RP3betaRecommender: Similarity column 27968 (100.0%), 1354.09 column/sec. Elapsed time 20.65 sec


In [26]:
# Instantiate and fit hybrid recommender
recommender = HybridRecommender_5(URM_train_aug, URM_train_aug,RP3beta_pow, S_SLIM, EASE_R, UserKNNCF)
#recommender = HybridRecommender(URM_train)
recommender.fit(UserKNNCF_tier1_weight=6345269660425519, RP3beta_pow_tier1_weight= 0.33158219696928976, EASE_R_tier1_weight=0.09597531837611298, UserKNNCF_tier2_weight=0.9792376938449392, RP3beta_pow_tier2_weight=0.570238999126259, EASE_R_tier2_weight=0.38860109710692725, RP3beta_pow_tier3_weight=0.722924027983384, S_SLIM_tier3_weight=0.8268022628613843, EASE_R_tier3_weight=0.06536164657983635, S_SLIM_tier4_weight= 0.9465915892992056, EASE_R_tier4_weight= 0.4292689877661564)
#'UserKNNCF_tier1_weight': 0.6345269660425519, 'RP3beta_pow_tier1_weight': 0.33158219696928976, 'EASE_R_tier1_weight': 0.09597531837611298, 'UserKNNCF_tier2_weight': 0.9792376938449392, 'RP3beta_pow_tier2_weight': 0.570238999126259, 'EASE_R_tier2_weight': 0.38860109710692725, 'RP3beta_pow_tier3_weight': 0.722924027983384, 'S_SLIM_tier3_weight': 0.8268022628613843, 'EASE_R_tier3_weight': 0.06536164657983635, 'S_SLIM_tier4_weight': 0.9465915892992056, 'EASE_R_tier4_weight': 0.4292689877661564

Hybrid_Recommender_5: URM Detected 3461 (12.4%) items with no interactions.


In [27]:
# Create CSV for submission
f = open("submission.csv", "w+")
f.write("user_id,item_list\n")
recommended_items_for_each_user = {}
for user_id in tqdm(target):
    recommended_items = recommender.recommend(user_id, cutoff=10, remove_seen_flag=True)
    #recommended_items=impressions.update_ranking(user_id,recommended_items,dataReader)
    recommended_items_for_each_user[int(user_id)]=recommended_items
    well_formatted = " ".join([str(x) for x in recommended_items])
    f.write(f"{user_id}, {well_formatted}\n")

100%|██████████| 41116/41116 [00:57<00:00, 719.39it/s]


In [28]:
# Evaluare recommended items
map=evaluate(recommended_items_for_each_user,URM_validation,target)
print('MAP score: {}'.format(map))

100%|██████████| 41116/41116 [00:01<00:00, 37667.09it/s]

MAP score: 0.034669620779269356
